In [ ]:
import pandas as pd
import pandas_gbq
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm

In [ ]:
# Set variables
from config import *

<h2>Define various helper functions</h2>

In [ ]:
def extract_url(x):
    if x != None:
        return x.get('string_value').split('?')[0]
    else:
        return '(entrance)'

In [ ]:
def cell_extract(x, target_str):
    if x != None:
        return x.get(target_str)
    else:
        return x

<h2>Extract/Transform</h2>

In [ ]:
# Get GA4 data on relevant users from BQ
ga4_df = pd.read_gbq(ga4_query_str, project_id=bq_project_str, credentials=bq_credentials)

In [ ]:
ga4_df = ga4_df.drop_duplicates(['user_pseudo_id', 'event_name', 'event_timestamp'])

In [ ]:
# GA4 has a lot of nested data in the schema, this pulls out the pieces we're interested in
ga4_df['page_location'] = ga4_df['page_location'].apply(lambda x: extract_url(x))
ga4_df['page_referrer'] = ga4_df['page_referrer'].apply(lambda x: extract_url(x))

ga4_df['user_email'] = ga4_df['user_email'].apply(lambda x: cell_extract(x, 'string_value'))

In [ ]:
ga4_df['utm_campaign'] = ga4_df['traffic_source'].apply(lambda x: cell_extract(x, 'name'))
ga4_df['utm_medium'] = ga4_df['traffic_source'].apply(lambda x: cell_extract(x, 'medium'))
ga4_df['utm_source'] = ga4_df['traffic_source'].apply(lambda x: cell_extract(x, 'source'))

ga4_df['form_id'] = ga4_df['form_id'].apply(lambda x: cell_extract(x, 'string_value'))

In [ ]:
# Categorize pages into groupings we're interested in
ga4_df['booking_step'] = ''

for step in url_category_dict.keys():
    ga4_df['booking_step'] = np.where(ga4_df['page_location'].str.contains(url_category_dict.get(step)), step, ga4_df['booking_step'])

In [ ]:
ga4_df['event_name'] = np.where(ga4_df['event_name'] == 'generate_lead', ga4_df['event_name'] + '_' + ga4_df['form_id'], ga4_df['event_name'])

In [ ]:
ga4_emails_df = ga4_df[ga4_df['user_email'].notnull()][['user_email', 'user_pseudo_id']].drop_duplicates('user_email')
ga4_emails_dict = dict(zip(ga4_emails_df['user_pseudo_id'], ga4_emails_df['user_email']))
ga4_df['user_email'] = ga4_df['user_pseudo_id'].apply(lambda x: ga4_emails_dict.get(x))

In [ ]:
# Get HubSpot data on deal stages from our BigQuery clone
hs_df = pd.read_gbq(hs_query_str, project_id=bq_project_str, credentials=bq_credentials)

In [ ]:
# Add columns with binary indicators for if a deal became a purchase or qualified lead
hs_df['purchase'] = np.where(hs_df['deal_pipeline_id'] == '13508273', 1, 0)

hs_df['opportunity'] = np.where((hs_df['deal_opportunity_date'].notnull()
                                 | hs_df['deal_priority_date'].notnull()
                                 | hs_df['deal_followup_date'].notnull()), 1, 0)

<h2>Transform</h2>

Create a new dataframe with binary variables for user behavoir and outcomes

In [ ]:
user_emails_list = list(ga4_df[ga4_df['user_email'].notnull()]['user_email'].unique())

In [ ]:
booking_page_list = list(ga4_df[ga4_df['booking_step'] != '']['booking_step'].unique())

In [ ]:
event_list = [x for x in list(ga4_df['event_name'].unique()) if x not in (session_stats_list + [np.nan, 'purchase'])]

In [ ]:
user_outcomes_df = pd.DataFrame()
user_outcomes_df = pd.DataFrame(columns=(['property_email'] + session_stats_list + ['utm_source', 'utm_campaign'] + booking_page_list + event_list))

In [ ]:
user_outcomes_df['property_email'] = user_emails_list

In [ ]:
user_outcomes_df = pd.merge(user_outcomes_df, hs_df[['property_email', 'opportunity', 'purchase']], 
                            how='left', on='property_email')

display(len(user_outcomes_df))

In [ ]:
for i in tqdm(user_outcomes_df.index):
    e = user_outcomes_df.loc[i, 'property_email']
    _df = ga4_df[ga4_df['user_email'] == e]
    
    for booking_step in booking_page_list:
        l = np.where(_df['booking_step'] == booking_step, 1, 0).sum()
        user_outcomes_df.loc[i, booking_step] = l
    
    for event in (event_list + session_stats_list):
        l = np.where(_df['event_name'] == event, 1, 0).sum()
        user_outcomes_df.loc[i, event] = l
            
    s, c = _df[['utm_source', 'utm_campaign']][-1:].values[0]
    user_outcomes_df.loc[i, 'utm_source'] = s
    user_outcomes_df.loc[i, 'utm_campaign'] = c
    
for booking_step in booking_page_list:
    user_outcomes_df[booking_step] = np.where(user_outcomes_df[booking_step] > 1, 1, 0)
    
for event in event_list:
    user_outcomes_df[event] = np.where(user_outcomes_df[event] > 1, 1, 0)

user_outcomes_df['session_start'] = user_outcomes_df['session_start'].astype('int64')
user_outcomes_df['page_view'] = user_outcomes_df['page_view'].astype('int64')

user_outcomes_df['pages/session'] = user_outcomes_df['page_view'] / user_outcomes_df['session_start']

In [ ]:
user_outcomes_df.replace(np.inf, 0, inplace=True)

In [ ]:
scaler = StandardScaler()
user_outcomes_df[['session_start', 'page_view', 'pages/session']] = scaler.fit_transform(user_outcomes_df[['session_start', 'page_view', 'pages/session']])

In [ ]:
source_dummies_df = pd.get_dummies(user_outcomes_df['utm_source'])
top_sources_list = list(source_dummies_df.sum().sort_values(ascending=False)[:10].index)
source_dummies_df = source_dummies_df[top_sources_list]
source_dummies_df.columns = ['source_' + x for x in list(source_dummies_df)]

In [ ]:
campaign_dummies_df = pd.get_dummies(user_outcomes_df['utm_campaign'])
top_campaigns_list = list(campaign_dummies_df.sum().sort_values(ascending=False)[:10].index)
campaign_dummies_df = campaign_dummies_df[top_campaigns_list]
campaign_dummies_df.columns = ['campaign_' + x for x in list(campaign_dummies_df)]

In [ ]:
user_outcomes_df = user_outcomes_df.fillna(0)

In [ ]:
user_outcomes_df = pd.concat([user_outcomes_df, source_dummies_df, campaign_dummies_df], axis=1)

In [ ]:
plt.figure(figsize=(16, 9))
heatmap = sns.heatmap(user_outcomes_df.corr()[['opportunity', 'purchase']], vmin=-1, vmax=1, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize': 18}, pad=12)

In [ ]:
user_outcomes_df.corr()[['opportunity', 'purchase']].sort_values('opportunity', ascending=False)

In [ ]:
153/4685

<h2>Modeling</h2>

In [ ]:
X_df = user_outcomes_df.drop(['property_email', 'utm_source', 'utm_campaign', 'opportunity', 'purchase'], axis=1)

In [ ]:
y_df = user_outcomes_df['opportunity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, train_size=0.7, test_size=0.3, random_state=3892)

In [ ]:
X_train = X_df
y_train = y_df

y_train.sum()

In [ ]:
#X_train.to_csv(r'x-train.csv')
#X_test.to_csv(r'x-test.csv')
#y_train.to_csv(r'y-train.csv')
#y_test.to_csv(r'y-test.csv')

In [ ]:
#X_train = pd.read_csv(r'x-train.csv')
#X_test = pd.read_csv(r'x-test.csv')
#y_train = pd.read_csv(r'y-train.csv')
#y_test = pd.read_csv(r'y-test.csv')

<h3>Model 1 - All variables</h3>

In [ ]:
logistic_m1_fitted = sm.Logit(y_train, X_train).fit(maxiter=100, method='ncg')
logistic_m1_fitted.summary()

<h3>Narrow variables with recursive factor elimination</h3>

In [ ]:
sk_log = LogisticRegression()
rfe = RFE(sk_log, step=2)
rfe = rfe.fit(X_train, y_train)

In [ ]:
predictor_cols = X_train.columns[rfe.support_]
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

<h3>Model 2 - Variables Identified by RFE</h3>

In [ ]:
logistic_m2_fitted = sm.Logit(y_train, X_train[predictor_cols]).fit(maxiter=100, method='powell')
logistic_m2_fitted.summary()

In [ ]:
m2_y_train_pred = logistic_m2_fitted.predict(X_train[predictor_cols])
m2_y_train_pred = m2_y_train_pred.values.reshape(-1)

In [ ]:
m2_y_train_pred = pd.DataFrame(index=y_train.index, data={'opportunity-actual':y_train.values, 'model-prob':m2_y_train_pred})
m2_y_train_pred['opportunity-model'] = m2_y_train_pred['model-prob'].apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
display(m2_y_train_pred['opportunity-actual'].sum())
display(m2_y_train_pred['opportunity-model'].sum())
m2_y_train_pred.sort_values('model-prob', ascending=False)

<h3>Model 3 - Web Events Only</h3>

In [ ]:
web_event_cols = [x for x in list(X_train) if 'source_' not in x and 'campaign_' not in x]

In [ ]:
logistic_m3_fitted = sm.Logit(y_train, X_train[web_event_cols]).fit(maxiter=100, method='ncg')
logistic_m3_fitted.summary()

In [ ]:
m3_y_train_pred = logistic_m3_fitted.predict(X_train[web_event_cols])
m3_y_train_pred = m3_y_train_pred.values.reshape(-1)

In [ ]:
m3_y_train_pred = pd.DataFrame(index=y_train.index, data={'opportunity-actual':y_train.values, 'model-prob':m3_y_train_pred})
m3_y_train_pred['opportunity-model'] = m3_y_train_pred['model-prob'].apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
display(m3_y_train_pred['opportunity-actual'].sum())
display(m3_y_train_pred['opportunity-model'].sum())
m3_y_train_pred.sort_values('model-prob', ascending=False)

<h3>Model 4 - Origins Only</h3>

In [ ]:
origins_cols = [x for x in list(X_train) if 'source_' in x or 'campaign_' in x]

In [ ]:
logistic_m4_fitted = sm.Logit(y_train, X_train[origins_cols]).fit(maxiter=100, method='lbfgs')
logistic_m4_fitted.summary()

In [ ]:
m4_y_train_pred = logistic_m4_fitted.predict(X_train[origins_cols])
m4_y_train_pred = m4_y_train_pred.values.reshape(-1)

In [ ]:
m4_y_train_pred = pd.DataFrame(index=y_train.index, data={'opportunity-actual':y_train.values, 'model-prob':m4_y_train_pred})
m4_y_train_pred['opportunity-model'] = m4_y_train_pred['model-prob'].apply(lambda x: 1 if x > 0.5 else 0)

In [ ]:
display(m4_y_train_pred['opportunity-actual'].sum())
display(m4_y_train_pred['opportunity-model'].sum())
m4_y_train_pred.sort_values('model-prob', ascending=False)

In [ ]:
vif = pd.DataFrame()
vif['features'] = X_train[web_event_cols].columns
vif['vif'] = [variance_inflation_factor(X_train[web_event_cols].values, i) for i in range(X_train[web_event_cols].shape[1])]
vif['vif'] = round(vif['vif'], 2)
vif = vif.sort_values(by = 'vif', ascending = False)
vif